In [1]:
import cv2 as cv
import sys
import time
import math
import numpy as np
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from collections import Counter

In [2]:
class CDI_lib():
    def __init__(self,istream,Threshold=True,Mask=False,mythold=[0,255]):
        super(CDI_lib, self).__init__()
        self.istream=istream
        #self.start_frame_num=start_frame_num
        self.Threshold=Threshold
        self.mythold=mythold
        self.Mask = Mask
    def CDI(self,start_frame_num,image_type='GRAY',methods='GRAY'):
        self.istream.set(cv.CAP_PROP_POS_FRAMES, start_frame_num)
        _, pre_frame = self.istream.read()
        _, frame = self.istream.read()
        _, next_frame = self.istream.read() 
        pre_frame = cv.resize(pre_frame, (224, 224),interpolation=cv.INTER_CUBIC)
        frame = cv.resize(frame, (224, 224),interpolation=cv.INTER_CUBIC)
        next_frame = cv.resize(next_frame, (224, 224),interpolation=cv.INTER_CUBIC)
        if image_type=='GRAY':
            pre_dst=cv.cvtColor(pre_frame, cv.COLOR_BGR2GRAY)
            dst=cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
            next_dst=cv.cvtColor(next_frame, cv.COLOR_BGR2GRAY)
        elif image_type=='BGR':
            pre_dst=pre_frame
            dst=frame
            next_dst=next_frame
        elif image_type=='HSV':
            pre_dst=cv.cvtColor(pre_frame, cv.COLOR_BGR2HSV)
            dst=cv.cvtColor(frame, cv.COLOR_BGR2HSV)
            next_dst=cv.cvtColor(next_frame, cv.COLOR_BGR2HSV)

        if methods == 'HARRIS':
            pre_dst = cv.cornerHarris(pre_dst,2,3,0.04)
            dst = cv.cornerHarris(dst,2,3,0.04)
            next_dst = cv.cornerHarris(next_dst,2,3,0.04)
        
        if methods == 'EDGE':
            pre_dst = cv.Canny(pre_dst,50,200)
            dst = cv.Canny(dst,50,200)
            next_dst = cv.Canny(next_dst,50,200)
        
        if methods == 'OPTICALFLOW':
            _, forth_frame = self.istream.read() 
            forth_frame = cv.resize(forth_frame, (224, 224),interpolation=cv.INTER_CUBIC)
            forth_dst=cv.cvtColor(forth_frame, cv.COLOR_BGR2GRAY)
            pre_dst = cv.calcOpticalFlowFarneback(pre_dst,dst, None, 0.5, 3, 15, 3, 5, 1.2, 0)
            dst = cv.calcOpticalFlowFarneback(dst,next_dst, None, 0.5, 3, 15, 3, 5, 1.2, 0)
            next_dst = cv.calcOpticalFlowFarneback(next_dst,forth_dst, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        
        if methods == 'SALIENCY':
            saliency = cv.saliency.StaticSaliencySpectralResidual_create()
            _,pre_dst = saliency.computeSaliency(pre_dst)
            pre_dst = (pre_dst * 255).astype("uint8")
            _,dst = saliency.computeSaliency(dst)
            dst = (dst * 255).astype("uint8")
            _,next_dst = saliency.computeSaliency(next_dst)
            next_dst = (next_dst * 255).astype("uint8")
        
        
        
        diff=cv.absdiff(pre_dst, dst)
        diff2=cv.absdiff(dst, next_dst)
        
        CDI_IMAGE=cv.bitwise_and(diff,diff2)
        
        # eliminate noise based on finding the contours of top 10 max areas, fill the contour to get more precise moving object description
        if self.Mask:
            mask = np.zeros_like(CDI_IMAGE)
            contours, _ = cv.findContours(CDI_IMAGE, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
            area = []
            for k in range(len(contours)):
                area.append(cv.contourArea(contours[k]))
            for f in range(10):
                if len(area) == 0:
                    break
                max_idx = np.argmax(np.array(area))
                cv.drawContours(mask, contours, max_idx, 255, cv.FILLED)
                del area[max_idx]
            CDI_IMAGE = mask
            
        if self.Threshold:
            
            _,diff=cv.threshold(diff, self.mythold[0], self.mythold[1], cv.THRESH_TOZERO)
            _,diff2=cv.threshold(diff2,self.mythold[0],self.mythold[1],cv.THRESH_TOZERO)
            CDI_IMAGE=cv.bitwise_and(diff,diff2)

        return CDI_IMAGE,pre_frame
    @staticmethod 
    def Pos_Morph(Image,OPEN_CLOSE=False,DILATE_ERODE=True,size1=1,size2=1):
        if DILATE_ERODE:
            kernel1 = cv.getStructuringElement(cv.MORPH_RECT,(size1,size1))
            kernel2 = cv.getStructuringElement(cv.MORPH_RECT,(size2,size2))
            Image = cv.erode(Image,kernel1)
            Image =cv.dilate(Image,kernel2)
        elif OPEN_CLOSE:
            kernel1 = cv.getStructuringElement(cv.MORPH_RECT,(size1,size1))
            kernel2 = cv.getStructuringElement(cv.MORPH_RECT,(size2,size2))
            Image = cv.morphologyEx(Image, cv.MORPH_OPEN, kernel1)
            Image = cv.morphologyEx(Image, cv.MORPH_CLOSE, kernel2)
        return Image
    @staticmethod 
    def CDI_Intensity(Image,Normalization=True,Entropy=True):
        
        if Entropy == True:
            if Normalization:
                dst=np.zeros(Image.shape)
                Image=cv.normalize(Image,dst,0,1,cv.NORM_MINMAX)
                
            #calculate entropy
            img_size = Image.shape[0] * Image.shape[1]
            histogram = Counter(Image.reshape(img_size))
            Intensity = 0
            for vals in histogram.values():
                vals = vals/img_size
                Intensity -= vals*(math.log(vals,2))

        else:        
            Intensity=np.sum(Image,axis=(1,0))
        #Intensity=np.sum(Image)
        return Intensity

    